In [17]:
import pandas as pd
df = pd.read_excel('panel.xlsx')
df = df.drop(columns=['Unnamed: 0'])
df = df.set_index(['region', 'period'])
df = df.map(lambda x: x.replace(',', '.') if isinstance(x, str) else x)
df = df.map(lambda x: x.replace(' ', '') if isinstance(x, str) else x)
df

KeyError: "['Unnamed: 0'] not found in axis"

In [9]:
df.dtypes

innovation_activity_level_percent           float64
labor_productivity_index_percent            float64
poverty_level_percent                       float64
industrial_production_index_percent         float64
construction_work_volume_thousand_rubles    float64
                                             ...   
elderly_people                                int64
elderly_and_teenagers                         int64
working_age_population                        int64
teenagers                                   float64
working_population                            int64
Length: 89, dtype: object

In [10]:
df.to_excel('panel.xlsx')

In [7]:
import pandas as pd

def find_non_numeric(df):
    non_numeric_cells = []
    for col in df.columns:
        for idx, value in df[col].items():
            try:
                pd.to_numeric(value)
            except ValueError:
                non_numeric_cells.append((col, idx, value))
    return non_numeric_cells

non_numeric_cells = find_non_numeric(df)
non_numeric_cells

[]

In [41]:
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
from linearmodels.panel import PanelOLS, RandomEffects

# Sample panel data
# data = {
#     'entity': ['A', 'A', 'A', 'B', 'B', 'B', 'C', 'C', 'C'],
#     'time': [1, 2, 3, 1, 2, 3, 1, 2, 3],
#     'variable1': [1, 2, 3, 2, 3, 4, 3, 4, 5],
#     'variable2': [2, 3, 4, 3, 4, 5, 4, 5, 6],
#     'variable3': [5, 3, 1, 4, 2, 0, 3, 1, -1]
# }
# 
# df = pd.DataFrame(data)
df = pd.read_excel('panel.xlsx')
df = df.set_index(['region', 'period'])
df = df.ffill()
# Compute correlation matrix
correlation_matrix = df[df.columns.tolist()].corr()
print("Correlation Matrix:")
print(correlation_matrix)

# Calculate VIF
df_with_const = sm.add_constant(df[df.columns.tolist()])
vif = pd.DataFrame()
vif['variable'] = df_with_const.columns
vif['VIF'] = [variance_inflation_factor(df_with_const.values, i) for i in range(df_with_const.shape[1])]
vif_repl = vif.loc[vif['VIF'] > 10]['variable'].tolist()[1:]
vif_repl_d = vif.loc[vif['VIF'] < 10]['variable'].tolist()
print("\nVIF:")

# Addressing multicollinearity using PCA
pca = PCA(n_components=1)
df['pca_component'] = pca.fit_transform(df[vif_repl])
df_reduced = df.drop(columns=vif_repl)

# Example Ridge regression to handle multicollinearity
X = df_reduced[['pca_component'] + vif_repl_d]
y = np.random.rand(len(df_reduced))  # Replace with your target variable

ridge = Ridge(alpha=1.0)
ridge.fit(X, y)
print("\nRidge Regression Coefficients:")
print(ridge.coef_)


model_fe = PanelOLS(y, X, entity_effects=True, check_rank=False, drop_absorbed=True)
results_fe = model_fe.fit()
print(results_fe.summary)

Correlation Matrix:
                                          innovation_activity_level_percent  \
innovation_activity_level_percent                                  1.000000   
labor_productivity_index_percent                                   0.023238   
poverty_level_percent                                             -0.347079   
industrial_production_index_percent                                0.009967   
construction_work_volume_thousand_rubles                           0.144239   
...                                                                     ...   
elderly_people                                                     0.292709   
elderly_and_teenagers                                              0.292935   
working_age_population                                             0.281178   
teenagers                                                          0.084565   
working_population                                                 0.277632   

                               

/home/chuckles/jup_dual/venv/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
/home/chuckles/jup_dual/venv/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.59652e-20): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T



VIF:

Ridge Regression Coefficients:
[-6.92623470e-12 -8.59889689e-04 -4.59395707e-03  2.19297035e-03
  6.08767497e-04  1.35191335e-04 -4.72115564e-03 -4.02243089e-03
  5.53335922e-02  8.96115922e-02 -8.54421056e-04  1.78423110e-01
  4.90321319e-03  1.08818954e-04  5.77724019e-03  3.05601257e-04
 -2.32556652e-04 -1.56551067e-06]


ValueError: 2 or 3-d array required for numpy input

In [36]:
vif.loc[vif['VIF'] > 10]['variable'].tolist()

['const',
 'poverty_level_percent',
 'construction_work_volume_thousand_rubles',
 'average_per_capita_income_rubles_per_month',
 'employed_population_thousands',
 'employment_level_percent',
 'increase_in_high_productivity_jobs_thousands',
 'active_enterprises',
 'closed_enterprises',
 'freight_turnover_million_ton_km',
 'commercial_freight_turnover_thousand_ton_km',
 'retail_trade_per_capita_rubles',
 'total_population_growth',
 'average_annual_population',
 'number_of_births_per_year',
 'age_specific_mortality_rate',
 'life_expectancy_at_birth_years',
 'total_fertility_rate',
 'birth_rate_age_15_19',
 'birth_rate_age_20_24',
 'birth_rate_age_25_29',
 'birth_rate_age_30_34',
 'birth_rate_age_35_39',
 'birth_rate_age_40_44',
 'number_of_deaths_per_year',
 'average_monthly_nominal_wage_rubles',
 'potential_workforce_thousands',
 'unemployment_and_potential_workforce_percent',
 'income_deficit',
 'arrivals_within_russia',
 'international_arrivals',
 'departures_within_russia',
 'internat

In [14]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Add a constant term for VIF calculation
df_with_const = sm.add_constant(df[['variable1', 'variable2', 'variable3']])

# Calculate VIF
vif = pd.DataFrame()
vif['variable'] = df_with_const.columns
vif['VIF'] = [variance_inflation_factor(df_with_const.values, i) for i in range(df_with_const.shape[1])]
vif

NameError: name 'sm' is not defined